# Setup

In [1]:
cd("../..")

# Packages

In [2]:
include("src/Utils.jl")
using .Utils

In [3]:
using Distributions, LinearAlgebra

In [4]:
using StatsBase, DataFrames

In [5]:
using RCall
R"""
library(rethinking)
""";

┌ Warning: RCall.jl: Loading required package: rstan
│ Loading required package: StanHeaders
│ Loading required package: ggplot2
│ rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)
│ For execution on a local, multicore CPU with excess RAM we recommend calling
│ options(mc.cores = parallel::detectCores()).
│ To avoid recompilation of unchanged Stan programs, we recommend calling
│ rstan_options(auto_write = TRUE)
│ Loading required package: parallel
│ Loading required package: dagitty
│ rethinking (Version 2.00)
│ 
│ Attaching package: ‘rethinking’
│ 
│ The following object is masked from ‘package:stats’:
│ 
│     rstudent
│ 
└ @ RCall /home/user/.julia/packages/RCall/paaBQ/src/io.jl:113


# Data

In [6]:
df = get_data("milk")
first(df[!, [:clade, Symbol("kcal.per.g")]], 6)

,clade,kcal.per.g
,String,Float64
1,Strepsirrhine,0.49
2,Strepsirrhine,0.51
3,Strepsirrhine,0.46
4,Strepsirrhine,0.48
5,Strepsirrhine,0.6
6,New World Monkey,0.47


How many unique clade are there:

In [7]:
unique(df.clade) |> sort

4-element Array{String,1}:
 "Ape"
 "New World Monkey"
 "Old World Monkey"
 "Strepsirrhine"

# Model

Predict `kcal.per.g` as a function of `clade`.

## One Hot Encoded

One hot encode the `clade` column. 

Let `Ape` be the default value ie. `isNWO` = `isOWM` = `isS` = 0.

In [8]:
ohe_clade = indicatormat(df.clade) |> 
    transpose |>
    (m -> DataFrame(m, [:isApe, :isNWO, :isOWM, :isS])) |>
    (df -> select(df, Not(:isApe)))

first(ohe_clade, 6)

,isNWO,isOWM,isS
,Bool,Bool,Bool
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
5,0,0,1
6,1,0,0


In [9]:
vars = (VariableSpecification(0, 100, Normal(0.6, 10)),
        VariableSpecification(-10, 10, Normal(0, 1)),
        VariableSpecification(-10, 10, Normal(0, 1)),
        VariableSpecification(-10, 10, Normal(0, 1)),
        VariableSpecification(0, 10, Uniform(0, 10)))
;

In [10]:
build_ll_data(data) = (α, β_NWO, β_OWM, β_S, σ) -> begin
    logprobs = map(data) do (isNWO, isOWM, isS, kcal)
        μ = α + β_NWO * isNWO + β_OWM * isOWM + β_S * isS
        d = Normal(μ, σ)
        logpdf(d, kcal)
    end
    
    logprobs |> sum
end;

ll_data = build_ll_data(zip(ohe_clade.isNWO,
                            ohe_clade.isOWM,
                            ohe_clade.isS,
                            df[!, Symbol("kcal.per.g")]));

In [11]:
build_l_joint_priors(priors::NTuple{5, Distribution}) = 
    (α, β_NWO, β_OWM, β_S, σ) -> logpdf.(priors, (α, β_NWO, β_OWM, β_S, σ)) |> sum

l_joint_priors = build_l_joint_priors(map(v -> v.prior, vars));

In [12]:
# the objective function
f(α, β_NWO, β_OWM, β_S, σ) = ll_data(α, β_NWO, β_OWM, β_S, σ) + l_joint_priors(α, β_NWO, β_OWM, β_S, σ);
f(xs::Vector) = f(xs...)

f (generic function with 2 methods)

In [13]:
soln_ohe, covarmat_ohe = quap(f, vars);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equ

In [14]:
round.(soln_ohe, digits=2)

5-element Array{Float64,1}:
  0.55
  0.17
  0.24
 -0.04
  0.11

Standard deviation of each parameter:

In [15]:
round.(diag(covarmat_ohe) .|> sqrt, digits=2)

5-element Array{Float64,1}:
 0.04
 0.05
 0.06
 0.06
 0.01

Note that the values of $\beta_\text{NWO}, \beta_\text{OWM}$ and $\beta_\text{S}$ are relative to `Ape`

We need to do some computation to figure out the absolute values of `kcal.per.g` for each species.

To get just the average value for a species, we can just add 0.55 (the coefficient for `Ape`) witht the $\beta$ for that species. To compute the uncertainty, you wil have to work with the posterior distribution.

## Index Variables

In [16]:
first(df, 5)

,clade,species,kcal.per.g,perc.fat,perc.protein,perc.lactose
,String,String,Float64,Float64,Float64,Float64
1,Strepsirrhine,Eulemur fulvus,0.49,16.6,15.42,67.98
2,Strepsirrhine,E macaco,0.51,19.27,16.91,63.82
3,Strepsirrhine,E mongoz,0.46,14.11,16.85,69.04
4,Strepsirrhine,E rubriventer,0.48,14.91,13.18,71.91
5,Strepsirrhine,Lemur catta,0.6,27.28,19.5,53.22


In [17]:
vars = (VariableSpecification(-10, 10, Normal(0, 1)),
        VariableSpecification(-10, 10, Normal(0, 1)),
        VariableSpecification(-10, 10, Normal(0, 1)),
        VariableSpecification(-10, 10, Normal(0, 1)),
        VariableSpecification(0, 10, Uniform(0, 10)))
;

In [18]:
build_ll_data(data) = (a1, a2, a3, a4, σ) -> begin
    clade_map = Dict("Ape" => a1,
                 "New World Monkey" => a2,
                 "Old World Monkey" => a3,
                 "Strepsirrhine" => a4)
    
    logprobs = map(data) do (clade, kcal)
        μ = clade_map[clade]
        d = Normal(μ, σ)
        logpdf(d, kcal)
    end
    
    logprobs |> sum
end;

ll_data = build_ll_data(zip(df.clade,
                            df[!, Symbol("kcal.per.g")]));

In [19]:
build_l_joint_priors(priors::NTuple{5, Distribution}) = 
    (a1, a2, a3, a4, σ) -> logpdf.(priors, (a1, a2, a3, a4, σ)) |> sum

l_joint_priors = build_l_joint_priors(map(v -> v.prior, vars));

In [20]:
# the objective function
f(a1, a2, a3, a4, σ) = ll_data(a1, a2, a3, a4, σ) + l_joint_priors(a1, a2, a3, a4, σ);
f(xs::Vector) = f(xs...)

f (generic function with 2 methods)

In [21]:
soln_iv, covarmat_iv = quap(f, vars);

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [22]:
round.(soln_iv, digits=2)

5-element Array{Float64,1}:
 0.54
 0.71
 0.79
 0.51
 0.11

In [23]:
round.(diag(covarmat_iv) .|> sqrt, digits=2)

5-element Array{Float64,1}:
 0.04
 0.04
 0.05
 0.05
 0.01

Verify that solution from one-hot encoding is identical with index variable approach:

In [24]:
ape = soln_ohe[1]
others = soln_ohe[2:4]
sigma_soln = soln_ohe[end]

vcat(ape, others .+ ape, sigma_soln) .|>
    arr -> round(arr, digits=2)

5-element Array{Float64,1}:
 0.55
 0.71
 0.79
 0.51
 0.11